<a href="https://colab.research.google.com/github/daniel-hain/bibliometrics_EIST_2021/blob/master/data/BERTopic_ST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial** - Topic Modeling with BERTopic


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

In [ ]:
import pandas as pd

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [ ]:
docs = pd.read_csv("https://raw.githubusercontent.com/daniel-hain/bibliometrics_EIST_2021/master/data/data_text.csv")

# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [ ]:
from bertopic import BERTopic

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("allenai-specter", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

In [ ]:
#topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(docs.loc[:,'text'])

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

2023-01-25 15:29:36,420 - BERTopic - Transformed documents to Embeddings
2023-01-25 15:29:39,853 - BERTopic - Reduced dimensionality
2023-01-25 15:29:39,917 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name
0,0,433,0_the_and_of_to
1,1,46,1_the_of_and_to


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

[('the', 0.13187838803461532),
 ('and', 0.10757536682306736),
 ('of', 0.10576891937523167),
 ('to', 0.07737308040264473),
 ('in', 0.07480863688948203),
 ('that', 0.04334699895095259),
 ('for', 0.041759191022960246),
 ('this', 0.04032200789269124),
 ('on', 0.03773504802120426),
 ('is', 0.03555231795396779)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [ ]:
### Attributes

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

ValueError: ignored

## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [ ]:
topic_model.visualize_heatmap(n_clusters=5, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs.loc[:,'text'], n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('the', 0.04816012202409878),
 ('of', 0.043975136685373416),
 ('and', 0.04128465599762049),
 ('to', 0.03931447272481977),
 ('in', 0.032798677942728754),
 ('that', 0.02709315547874713),
 ('sharing', 0.02653621341170918),
 ('is', 0.024146796282296268),
 ('we', 0.023152455249453918),
 ('electric', 0.022051101320927056)]

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs.loc[:,'text'], nr_topics=60)

2023-01-25 15:21:55,608 - BERTopic - Reduced number of topics from 12 to 12


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[1, 0, 1, -1, 1, -1, -1, 0, 10, -1, 3, 7, -1, 0, -1, -1, -1, 1, 0, 0, -1, 10, 3, 8, -1, -1, 1, 4, 2, -1, 5, 10, 3, -1, 0, -1, 1, -1, -1, 1, 4, 0, 7, 0, -1, 1, 6, 2, -1, 6, -1, 3, 8, 4, 1, 3, 10, 5, 2, -1, 0, 3, -1, 0, 8, 4, 0, -1, 3, 6, 8, -1, 0, 2, 0, 1, -1, -1, -1, 0, 5, -1, 9, 6, 3, -1, 5, 1, -1, 1, 0, 3, 7, 1, 1, -1, 9, 0, 0, 2, 2, -1, -1, 0, 3, 1, 3, 0, -1, -1, 8, -1, 6, 4, -1, -1, 8, 3, 0, 0, -1, 10, -1, -1, -1, 3, 7, 1, 0, 6, 5, 2, 0, 4, 0, -1, 6, 5, -1, 2, 0, -1, 0, -1, 1, 4, 8, 0, -1, 5, -1, 3, 6, -1, 3, -1, 0, 1, 1, 1, 1, 4, -1, -1, 0, 0, -1, 9, 7, 1, 4, -1, 2, 2, 7, 10, 5, 1, -1, 5, -1, 3, 2, 0, -1, 0, 3, 4, -1, -1, -1, 7, 0, 0, 4, 2, 3, 0, 3, -1, -1, 3, 1, 7, -1, -1, 5, 5, 2, 3, -1, -1, 1, 10, 5, 10, 3, -1, 2, -1, 10, 3, 3, 5, 7, 3, 3, 9, 4, -1, 1, 10, -1, 0, -1, -1, -1, 0, 1, -1, 1, 6, -1, 0, 2, -1, 9, 5, -1, -1, -1, 0, -1, 4, -1, -1, -1, -1, -1, 9, -1, 2, 4, -1, -1, 3, 3, 0, 1, -1, -1, 2, 3, -1, -1, 0, 3, -1, -1, 9, 3, 4, -1, -1, -1, -1, 0, 1, -1, 5, 0, 3, -1, 5, -1, 5, 0

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [ ]:
similar_topics, similarity = topic_model.find_topics("technological change", top_n=5); similar_topics

[1, 8, 3, 10, 7]

In [ ]:
topic_model.get_topic(10)

[('of', 0.04157320569174931),
 ('the', 0.041491375369194915),
 ('and', 0.03322855851688495),
 ('niche', 0.02802054731810247),
 ('to', 0.024169592698409962),
 ('in', 0.023598416068200446),
 ('systems', 0.02284685914241872),
 ('innovation', 0.02173482308193439),
 ('by', 0.021638358914215016),
 ('this', 0.01804342571933703)]

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [ ]:
# Save model
topic_model.save("my_model")	

In [ ]:
# Load model
my_model = BERTopic.load("my_model")	